<a href="https://colab.research.google.com/github/HueyVault/study_NLPs/blob/main/quests/DeepLearnings/linearregression_house_prices_advanced_regression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 수집


In [172]:
import os
# 환경 변수에 따라 파일 경로 설정
if os.getenv('KAGGLE_KERNEL_RUN_TYPE') is not None:
    # Kaggle 환경
    train_file_path =  f'/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
    test_file_path = f'/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
    print("kaggle")
else:
    # Docker 환경
    train_file_path = "../../datasets/train.csv"
    test_file_path = "../../datasets/test.csv"
    print("another")
    

kaggle


In [173]:
import pandas as pd

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [174]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [175]:
test_df.shape

(1459, 80)

In [176]:
def get_continuous_columns(df, unique_threshold=10):
    """
    DataFrame에서 연속형 수치 변수를 찾는 함수
    
    Parameters:
    -----------
    df : pandas DataFrame
        분석할 데이터프레임
    unique_threshold : int, default=10
        연속형으로 간주할 최소 unique 값 개수
        
    Returns:
    --------
    list : 조건을 만족하는 컬럼명 리스트
    """
    
    continuous_columns = []
    
    for col in df.columns:
        # 1. object 타입이 아닌지 확인
        is_numeric = df[col].dtype != 'object'
        
        # 2. 결측치가 없는지 확인
        no_missing = df[col].isna().sum() == 0
        
        # 3. unique 값의 개수가 임계값보다 큰지 확인
        enough_unique = df[col].nunique() >= unique_threshold
        
        # 모든 조건을 만족하면 리스트에 추가
        if is_numeric and no_missing and enough_unique:
            print(df[col].nunique())
            continuous_columns.append(col)
    
    return continuous_columns

In [177]:
target_column = 'SalePrice'

continuous_cols = get_continuous_columns(train_df.drop(columns=['Id', target_column]), unique_threshold=50)
print("연속형 변수:", continuous_cols)

1073
112
61
637
144
780
721
753
417
861
441
274
202
120
76
연속형 변수: ['LotArea', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch']


In [178]:
feature_columns = continuous_cols # ['BsmtUnfSF','LowQualFinSF','HalfBath','Fireplaces']

select_df = train_df[feature_columns]
select_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   LotArea        1460 non-null   int64
 1   YearBuilt      1460 non-null   int64
 2   YearRemodAdd   1460 non-null   int64
 3   BsmtFinSF1     1460 non-null   int64
 4   BsmtFinSF2     1460 non-null   int64
 5   BsmtUnfSF      1460 non-null   int64
 6   TotalBsmtSF    1460 non-null   int64
 7   1stFlrSF       1460 non-null   int64
 8   2ndFlrSF       1460 non-null   int64
 9   GrLivArea      1460 non-null   int64
 10  GarageArea     1460 non-null   int64
 11  WoodDeckSF     1460 non-null   int64
 12  OpenPorchSF    1460 non-null   int64
 13  EnclosedPorch  1460 non-null   int64
 14  ScreenPorch    1460 non-null   int64
dtypes: int64(15)
memory usage: 171.2 KB



## 데이터 전처리
- 데이터 분석

In [179]:
from sklearn.preprocessing import StandardScaler


In [180]:
standardscaler_feature = StandardScaler()
standardscaler_feature.fit(select_df)

StandardScaler()

In [181]:
scailing_array_feature = standardscaler_feature.transform(select_df)

In [182]:
scailing_array_feature.shape, type(scailing_array_feature)

((1460, 15), numpy.ndarray)

In [183]:
y_train = train_df[target_column].values.reshape(-1, 1)  # 2차원 배열로 변환

y_train.shape

(1460, 1)

In [184]:
standardscaler_label = StandardScaler()
standardscaler_label.fit(y_train)

StandardScaler()

In [185]:
scailing_array_label = standardscaler_label.transform(y_train)

In [186]:
scailing_array_feature[5], scailing_array_label[5]

(array([ 0.3606161 ,  0.71978635,  0.49103992,  0.63244965, -0.28865283,
        -1.13928601, -0.59611547, -0.94869107,  0.50187469, -0.29214465,
         0.03284429, -0.43293146, -0.25153911, -0.3593249 , -0.27020835]),
 array([-0.47750496]))

## 데이터 분할
- train, test, validation

In [187]:
from sklearn.model_selection import train_test_split


In [188]:
train_features, test_features, train_label, test_label =\
      train_test_split(scailing_array_feature, scailing_array_label, test_size=0.2, random_state=42)
train_features.shape, test_features.shape, train_label.shape, test_label.shape

((1168, 15), (292, 15), (1168, 1), (292, 1))

In [189]:
import torch

In [190]:
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_label_tensor = torch.tensor(train_label, dtype=torch.float32).view(-1,1) # 행만 있는 경우 행열로 변환 해줌
# test_features_tensor = torch.tensor(test_features)
# test_label_tensor = torch.tensor(test_label)
type(train_features_tensor), type(train_label_tensor)

(torch.Tensor, torch.Tensor)

In [191]:
train_features_tensor.shape, train_label_tensor.shape

(torch.Size([1168, 15]), torch.Size([1168, 1]))

## 모델 학습



In [192]:
# Neural Network model linear regression
# model, loss function, optimizer function
class LinearRegressionNNM(torch.nn.Module) :
    def __init__(self, input_dim): # input : feature 의 열 갯수
        super(LinearRegressionNNM, self).__init__()
        # super(self).__init__()
        self.hidden_1 = torch.nn.Linear(input_dim, 64) # input_dim : feature 수 10개, output_dim : 출력 수 64 개
        self.hidden_2 = torch.nn.Linear(64, 32) # input_dim : 이전 layer output_dim 수 64개, output_dim : 출력 수 32 개
        self.hidden_3 = torch.nn.Linear(32, 16) # input_dim : 이전 layer output_dim 수 32, output_dim : 출력 수 16 개
        self.output = torch.nn.Linear(16, 1) # input_dim : 이전 layer output_dim 수 16, output_dim : 출력 수 1 개 Linear 니까
        self.relu = torch.nn.ReLU() # activation func ReLU 쓸거임. 

    def forward(self, x):
        x = self.relu(self.hidden_1(x)) # hidden layer 1 10,64
        x = self.relu(self.hidden_2(x)) # hidden layer 2 64,32
        x = self.relu(self.hidden_3(x)) # hidden layer 3 32,16
        out = self.output(x) # hidden layer 4 16,1 result 
        
        return out


In [193]:
train_features_tensor.shape[1], train_label_tensor.shape[1]

(15, 1)

In [194]:
model = LinearRegressionNNM(train_features_tensor.shape[1])

In [195]:
model

LinearRegressionNNM(
  (hidden_1): Linear(in_features=15, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [196]:
criterion = torch.nn.MSELoss() # Loss function 거의 label 타입에 따라 결정됨
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # Optimizer fuction
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Optimizer fuction

In [197]:
## 반복 학습
# loss 가 한자리 까지 줄어야 해
# for epoch in range(10):
for epoch in range(1000):
# for epoch in range(100000):
    pred_y = model.forward(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"epoch : {epoch}, loss : {loss.item()}")
    # print(f"epoch : {epoch}, loss : {loss.item()}")
    



epoch : 0, loss : 0.9476805329322815
epoch : 100, loss : 0.17867852747440338
epoch : 200, loss : 0.08934050053358078
epoch : 300, loss : 0.06379646062850952
epoch : 400, loss : 0.0514773465692997
epoch : 500, loss : 0.04194973409175873
epoch : 600, loss : 0.03419928997755051
epoch : 700, loss : 0.028756240382790565
epoch : 800, loss : 0.02483159489929676
epoch : 900, loss : 0.021743906661868095


## 모델 평가



In [198]:
model.eval()

LinearRegressionNNM(
  (hidden_1): Linear(in_features=15, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [199]:
with torch.no_grad(): # 학습 목정이 아닌 평가 목적 위해 고정
    pred_y = model(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor) # 예측도 간은 loss function
    print(f"loss : {loss.item()}")


loss : 0.01891646720468998


In [200]:
# 결과 값(예측값, 원래값)
pred_y[0], train_label[0]

(tensor([-0.5812]), array([-0.45232089]))

## 모델 배포

In [201]:
select_test_df = test_df[feature_columns]
select_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotArea        1459 non-null   int64  
 1   YearBuilt      1459 non-null   int64  
 2   YearRemodAdd   1459 non-null   int64  
 3   BsmtFinSF1     1458 non-null   float64
 4   BsmtFinSF2     1458 non-null   float64
 5   BsmtUnfSF      1458 non-null   float64
 6   TotalBsmtSF    1458 non-null   float64
 7   1stFlrSF       1459 non-null   int64  
 8   2ndFlrSF       1459 non-null   int64  
 9   GrLivArea      1459 non-null   int64  
 10  GarageArea     1458 non-null   float64
 11  WoodDeckSF     1459 non-null   int64  
 12  OpenPorchSF    1459 non-null   int64  
 13  EnclosedPorch  1459 non-null   int64  
 14  ScreenPorch    1459 non-null   int64  
dtypes: float64(5), int64(10)
memory usage: 171.1 KB


In [202]:
scailing_array_test = standardscaler_feature.transform(select_test_df)


In [203]:
test_features_tensor = torch.tensor(scailing_array_test, dtype=torch.float32)


In [204]:
with torch.no_grad(): # 학습 목정이 아닌 평가 목적 위해 고정
    pred_y = model(test_features_tensor)


In [205]:
import numpy as np
# inverse transform 및 반올림
pred_y_fin = np.round(standardscaler_label.inverse_transform(pred_y.reshape(-1, 1))).astype(int)

# 1차원 배열로 변환 (필요한 경우)
pred_y_fin = pred_y_fin.ravel()  # or pred_y_fin.flatten()

<ipython-input-205-7973d5aff73e>:3: RuntimeWarning: invalid value encountered in cast
  pred_y_fin = np.round(standardscaler_label.inverse_transform(pred_y.reshape(-1, 1))).astype(int)


In [206]:
output = pd.DataFrame({'Id': test_df.Id, 'SalePrice': pred_y_fin})
output.head()

,Id,SalePrice
0,1461,122318
1,1462,150242
2,1463,210893
3,1464,181476
4,1465,193752


In [207]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Id         1459 non-null   int64
 1   SalePrice  1459 non-null   int64
dtypes: int64(2)
memory usage: 22.9 KB


In [208]:
output.to_csv('submission.csv', index=False) # sample_submission

In [209]:
output.shape

(1459, 2)

In [210]:
test_df.shape

(1459, 80)